In [1]:
%matplotlib ipympl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import physoce.tseries as ts
from scipy.signal import welch
import gsw

In [2]:
current_file = '../data/Rover_II_Current_Mag_Hourly_Avg_pad_2018.csv'
df = pd.read_csv(current_file,parse_dates=[1])

In [3]:
df.head()

,Unnamed: 0,Date_time_Merge_hour,Current_mag,Easting (cm/sec),Northing (cm/sec),Minutes,Seconds,Date_time_R
0,1,2011-11-20 13:00:00,7.712734,-3.678000,-1.354000,10.8,32.0,11/20/2011 13:11
1,2,2011-11-20 14:00:00,1.711170,1.253333,-0.853333,16.0,13.0,11/20/2011 13:56
2,3,2011-11-20 15:00:00,2.024802,1.080000,-1.430000,25.0,30.0,11/20/2011 14:55
3,4,2011-11-20 16:00:00,3.567306,-1.413333,-3.036667,19.0,28.0,11/20/2011 15:59
4,5,2011-11-20 17:00:00,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df['Date_time_R'] = pd.to_datetime(df['Date_time_R'])

In [5]:
df = df.set_index('Date_time_R')

In [6]:
df = df.drop('Unnamed: 0', axis = 1)

In [7]:
#calculating speed for the whole data set
df['speed'] = np.sqrt((df['Easting (cm/sec)']**2) + (df['Northing (cm/sec)']**2))

In [8]:
#separating out the data from the year 2014-2018
#dfsub = df.loc['2014-09-01':'2018-12-31']

#dfsub = df.loc['2010-09-01':'2018-12-31']

dfsub = df.loc['2015-10-01':'2018-12-31']

#dfsub = df

In [9]:
#taking the monthy averages of the speed of the current 
dfsubmonthly = dfsub.resample('2m').mean()

#taking the monthy std dev of the speed of the current 
dfstdmonthly = dfsub.resample('2m').std()

In [10]:
plt.figure(figsize = (8,5))
plt.plot(dfsubmonthly.index, dfsubmonthly['Easting (cm/sec)'])
plt.xlabel('Time [year-month]')
plt.ylabel('East [cm/s]')

plt.figure(figsize = (8,5))
plt.plot(dfsubmonthly.index, dfsubmonthly['Northing (cm/sec)'])
plt.xlabel('Time [year-month]')
plt.ylabel('North [cm/s]')

FigureCanvasNbAgg()

FigureCanvasNbAgg()

Text(0, 0.5, 'North [cm/s]')

In [11]:
plt.figure(figsize = (8,5))
plt.plot(dfstdmonthly.index, dfstdmonthly['Easting (cm/sec)'])
plt.xlabel('Time [year-month]')
plt.ylabel('East [cm/s]')

plt.figure(figsize = (8,5))
plt.plot(dfstdmonthly.index, dfstdmonthly['Northing (cm/sec)'])
plt.xlabel('Time [year-month]')
plt.ylabel('North [cm/s]')

FigureCanvasNbAgg()

FigureCanvasNbAgg()

Text(0, 0.5, 'North [cm/s]')

In [12]:
dfhr = dfsub.resample('1H',limit=1).mean().interpolate(limit=3)

In [13]:
dfhr['uf'] = ts.pl64(dfhr['Easting (cm/sec)'])
dfhr['vf'] = ts.pl64(dfhr['Northing (cm/sec)'])

/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/scipy/signal/signaltools.py:491: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return x[reverse].conj()
/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/scipy/signal/signaltools.py:251: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  in1zpadded[sc] = in1.copy()


In [14]:
T_f_hrs = 1/(gsw.f(35+8.4585/60)/(2*np.pi))/3600

dfhr['ubp'] = ts.pl64(dfhr['Easting (cm/sec)'],T=T_f_hrs/1.00) - ts.pl64(dfhr['Easting (cm/sec)'],T=T_f_hrs/1.2) 
dfhr['vbp'] = ts.pl64(dfhr['Northing (cm/sec)'],T=T_f_hrs/1.00) - ts.pl64(dfhr['Northing (cm/sec)'],T=T_f_hrs/1.2) 

In [15]:
#print(hourly)
hourly_time_number = (dfhr.index-np.datetime64('2000-01-01T00:00'))/np.timedelta64(1,'D')

In [16]:
plt.figure()
plt.plot(dfhr.index,dfhr['Easting (cm/sec)'])
plt.plot(dfhr.index,dfhr['Northing (cm/sec)'])

FigureCanvasNbAgg()

In [17]:
plt.figure()
plt.plot(dfhr.index,dfhr['uf'])
plt.plot(dfhr.index,dfhr['vf'])
plt.xticks(rotation=30)

FigureCanvasNbAgg()

(array([735964., 736085., 736208., 736330., 736450., 736573., 736695.,
        736815., 736938.]), <a list of 9 Text xticklabel objects>)

In [18]:
plt.figure()
plt.plot(dfhr.index,dfhr['ubp'])
plt.plot(dfhr.index,dfhr['vbp'])
plt.xticks(rotation=30)

FigureCanvasNbAgg()

(array([735964., 736085., 736208., 736330., 736450., 736573., 736695.,
        736815., 736938.]), <a list of 9 Text xticklabel objects>)

In [19]:
plt.figure()
plt.plot(dfhr['ubp'],dfhr['vbp'])
plt.gca().axis('equal')

FigureCanvasNbAgg()

(-0.9963214527978606,
 0.9840904483252526,
 -0.8775212841550756,
 0.8940587419971519)

In [20]:
t1 = np.datetime64('2017-11-12T00:00:00')
t2 = np.datetime64('2018-10-17T00:00:00')

In [21]:
ti, = np.where((dfhr.index >= t1) & (dfhr.index <= t2))
dfhrsub = dfhr.loc[(dfhr.index >= t1) & (dfhr.index <= t2)]

In [22]:
Eas = np.array(dfhrsub['Easting (cm/sec)'])/100
Nor = np.array(dfhrsub['Northing (cm/sec)'])/100

N = len(Eas)
fE,SE = welch(Eas,fs=24.,nperseg=N/6,window='hanning')
fN,SN = welch(Nor,fs=24.,nperseg=N/6,window='hanning')

#rotary spectrum
fr,Sr = welch(Eas+1j*Nor,
              fs=24.,nperseg=N/6,
              window='hanning',return_onesided=False)
icw, = np.where(fr<0)
iccw, = np.where(fr>0)
fcw = -fr[icw]
Scw = Sr[icw]
fccw = fr[iccw]
Sccw = Sr[iccw]

In [23]:
# rotary coefficient
Cr = np.abs(Scw[1:]-Sccw[::-1])/(Scw[1:]+Sccw[::-1])
fr = fcw[1:]

In [24]:
f_cpd = gsw.f(35+8.4585/60)*86400/(2*np.pi)

# theoretical prediction (Van Haren and Millot (2004), eq. 11)
sigmar = 2*np.pi*fr
Cr_pred = 2*sigmar*(2*np.pi*f_cpd)/(sigmar**2 + (2*np.pi*f_cpd)**2)

In [25]:
plt.figure()
plt.loglog(fcw,Scw)
plt.loglog(fccw,Sccw,'--')
plt.legend(['clockwise','counter-clockwise'])
plt.xlabel('frequency [cpd]')
plt.ylabel('[m$^2$ s$^{-2}$ cpd$^{-1}$]')
plt.title('rotary spectra')

FigureCanvasNbAgg()

Text(0.5, 1.0, 'rotary spectra')

In [26]:
iwi, = np.where(fr >= f_cpd)

plt.figure()
plt.plot(fr,Cr)
plt.plot(fr[iwi],Cr_pred[iwi],'k-',lw=2)
yl = plt.ylim()
plt.plot([f_cpd,f_cpd],yl,'k--')
plt.xlabel('f [cpd]')
plt.ylabel('$C_R$')
plt.title('rotary coefficients')
plt.ylim(yl)
plt.xlim([0.6,10])
plt.ylim([0,1])

FigureCanvasNbAgg()

(0, 1)